In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from google.colab import files

data = []

for page_num in range(1, 81):
    url = f"https://www.zangia.mn/job/list/lmt.100/pg.{page_num}"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    job_listings = soup.select('.list div[class*=lvl]')
    print("page: ",page_num)

    jobCount = 0
    for job_listing in job_listings:
        company = job_listing.find(class_="company")
        companyTitle = company.find('b').string.strip()

        job = job_listing.find(class_="ad")
        jobTitle = job.find("b").string.strip()
        payRange = job.find(class_="fsal").string.strip()
        location_element = job.find(class_="floc")
        if location_element:
          location = location_element.string.strip()
        else:
          location = ''
    
        industry = ''
        jobType = ''
        level = ''
        jobRequirment = ''
        jobDescription = ''

        detailSlug = job.find('a').get('href')
        detailPage = requests.get(f"https://www.zangia.mn/{detailSlug}")
        detailSoup = BeautifulSoup(detailPage.content, "html.parser")

        details = detailSoup.find(class_='details')
        
        for div in details.find_all('div'):
          key = div.b.text.strip()
          value = div.span.text.strip()
          if key == 'Салбар':
            industry = value
          if key == 'Түвшин':
            level = value
          if key == 'Төрөл':
            jobType = value    

        
        for section in detailSoup.find_all(class_='section'):
          key = section.h2.text.strip()
          text_list = [p.text for p in section.div.find_all('p')]
          value = "\n".join(text_list)

          if key == 'Гүйцэтгэх үндсэн үүрэг':
            jobDescription = value
          if key == 'Ажлын байранд тавигдах шаардлага':
            jobRequirment = value

        jobCount += 1
        print(jobCount)
        if jobCount % 10 == 0:
          print(companyTitle, jobTitle, payRange, location, industry, jobType, level, jobDescription, jobRequirment)
        
        data.append([companyTitle, jobTitle, payRange, location, industry, jobType, level, jobDescription, jobRequirment])

      
with open("jobs_data.tsv", "w", newline="", encoding="utf-8") as f:
  writer = csv.writer(f, delimiter="\t")
  writer.writerow(["Company", "Job Title", "Pay Range", "Location", "Industry", "Job Type", "Level", "Job Description", "Job Requirement"])
  writer.writerows(data)
files.download("jobs_data.tsv")

Streaming output truncated to the last 5000 lines.
2. 3-с дээш жилийн ажлын туршлагатай
81
82
83
84
85
86
87
88
89
90
Новотел Улаанбаатар Зочид Буудал Цахилгаанчин 1,000,000 - 1,200,000 Тохиролцоно  Барилга, Үл хөдлөх хөрөнгө Ээлжийн Мэргэжилтэн - ИТА-ны өдөр тутмын цахилгааны ажлыг хийж гүйцэтгэх, төлөвлөгөөт болон төлөвлөгөөт бус засварын ажлыг хийж гүйцэтгэх
- Цахилгаан самбар болон тоног төхөөрөмжүүдэд тогтмол үзлэг, шалгалт хийх 
- Тухайн өдрийн хэрэглээний тоолуурын заалтыг өдөр бүр авах, тайлагнах 
- Цахилгааны талаас хэвийн үйл ажиллагааг хангах 
- Цахилгааны бүх самбар болон тоног төхөөрөмжийн хэвийн үйл ажиллагааг хангах
- Дизель генераторт үзлэг шалгалт хийх, хэвийн үйл ажиллагааг хангах  - Цахилгааны чиглэлээр Их дээд сургууль болон МСҮТ төгссөн байх
- Мэргэжлээрээ ажилласан туршлагатай байх
- Байгууллагын дотоод дүрэм, журам, аюулгүй ажиллагааг дагаж мөрдөх
- Хариуцлагатай, ээлжийн цагийн хуваарийн дагуу ажиллах боломжтой 
- Томоохон зочид буудал, ресторануудад ажиллаж бай

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import csv
from google.colab import files
import io
import matplotlib.pyplot as plt
import re

# Upload the file
uploaded = files.upload()

# Read the file into a pandas DataFrame
df = pd.read_csv(io.StringIO(uploaded['new-data.tsv'].decode('utf-8')), delimiter='\t')

# Extract Min Pay, Max Pay, and Average Pay from Pay Range
min_pay = []
max_pay = []
avg_pay = []
for pay_range in df['Pay Range']:
    matches = re.findall(r'\d+', pay_range)
    if matches:
        min_pay.append(int(matches[0]))
        max_pay.append(int(matches[-1]))
        avg_pay.append((int(matches[0]) + int(matches[-1])) / 2)
    else:
        min_pay.append(None)
        max_pay.append(None)
        avg_pay.append(None)

# Add the new fields to the DataFrame
df['Min Pay'] = min_pay
df['Max Pay'] = max_pay
df['Average Pay'] = avg_pay

# Write the updated DataFrame to a new file
df.to_csv('new-data-updated.tsv', index=False, sep='\t')

# Download the new file
files.download('new-data-updated.tsv')


Saving new-data.tsv to new-data (2).tsv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import csv
from google.colab import files
import io
import matplotlib.pyplot as plt
import re

# Upload the file
uploaded = files.upload()

# Read the file into a pandas DataFrame
df = pd.read_csv(io.StringIO(list(uploaded.values())[0].decode('utf-8')), delimiter='\t')

# Extract Min Pay, Max Pay, and Average Pay from Pay Range
min_pay = []
max_pay = []
avg_pay = []
for pay_range in df['Pay Range']:
    matches = re.findall(r'[\d,]+', pay_range)
    if matches:
        min_pay.append(int(matches[0].replace(',', '')))
        max_pay.append(int(matches[-1].replace(',', '')))
        avg_pay.append((int(matches[0].replace(',', '')) + int(matches[-1].replace(',', ''))) / 2)
    else:
        min_pay.append(None)
        max_pay.append(None)
        avg_pay.append(None)

# Add the new fields to the DataFrame
df['Min Pay'] = min_pay
df['Max Pay'] = max_pay
df['Average Pay'] = avg_pay

# Write the updated DataFrame to a new file
df.to_csv('data-updated.tsv', index=False, sep='\t')

# Download the new file
files.download('data-updated.tsv')


Saving data.tsv to data.tsv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from google.colab import files

data = []
url = f"https://www.zangia.mn/job/list/lmt.100/pg.{page_num}"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
job_listings = soup.select('.list div[class*=lvl]')
print("page: ",page_num)